# Feature Selection & Data Leakage Check

## 🎯 目标
1. **识别和分类所有145个特征**
2. **检测 Data Leakage 风险**
3. **生成特征分类清单**

## ⚠️ Data Leakage 是什么？

**Data Leakage** 是指训练数据中包含了**只有在预测时才会知道的信息**。

### 例子：
- ❌ **错误**: 使用 `total_pymnt`（总还款额）预测 `loan_status`
  - 问题：只有贷款完成后才知道总还款额
  - 结果：模型在训练时表现完美，但实际应用时无法获得这个特征

- ✅ **正确**: 使用 `annual_inc`（年收入）、`dti`（债务收入比）预测 `loan_status`
  - 原因：这些信息在贷款申请时就已知
  - 结果：模型可以真实反映预测能力

---

## 📋 特征分类标准

### ✅ **PRE-LOAN Features (可以使用)**
贷款申请时就已经知道的信息：
- 借款人基本信息
- 信用历史（申请前的）
- 贷款申请信息

### ❌ **POST-LOAN Features (必须删除 - 会造成 Leakage)**
贷款发放后才产生的信息：
- 还款记录
- 收回金额
- 未偿还本金

### 🗑️ **Metadata Features (需要删除 - 无预测价值)**
- ID 字段
- URL
- 地理位置细节

### 🎯 **Target Variable (目标变量)**
- `loan_status` - 我们要预测的变量

---

In [ ]:
# Cell 1: 导入库
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

print("✓ 库导入成功")

In [ ]:
# Cell 2: 加载数据
import os

# 优先加载 loan_with_desc.csv，如果不存在则加载 loan.csv
if os.path.exists('../../data/loan_with_desc.csv'):
    print("加载 loan_with_desc.csv...")
    df = pd.read_csv('../../data/loan_with_desc.csv', low_memory=False)
    print(f"✓ 已加载有desc的数据子集")
else:
    print("加载 loan.csv...")
    df = pd.read_csv('../../data/loan.csv', low_memory=False)
    # 过滤有desc的数据
    df = df[df['desc'].notna() & (df['desc'].astype(str).str.strip().str.len() > 1)].copy()
    print(f"✓ 已过滤出有desc的数据")

print(f"\n数据形状: {df.shape[0]:,} 行 × {df.shape[1]} 列")
print(f"\n所有列名:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:3d}. {col}")

---
## 📊 特征分类

现在我们将对所有特征进行详细分类。分类基于**时间顺序**：
- 贷款申请前就知道的信息 ➜ **PRE-LOAN**
- 贷款发放后才产生的信息 ➜ **POST-LOAN**
---

In [ ]:
# Cell 3: 定义特征分类

print("=" * 80)
print("特征分类定义")
print("=" * 80)

# ========================================================================
# ✅ PRE-LOAN FEATURES (贷款前 - 可以使用)
# ========================================================================
pre_loan_features = [
    # 1. 贷款申请基本信息
    'loan_amnt',           # 贷款金额
    'funded_amnt',         # 资助金额
    'funded_amnt_inv',     # 投资者资助金额
    'term',                # 期限
    'int_rate',            # 利率
    'installment',         # 分期付款额
    'grade',               # 等级
    'sub_grade',           # 子等级
    'purpose',             # 贷款目的
    'title',               # 贷款标题
    'desc',                # 贷款描述（用于OCEAN特征提取）
    
    # 2. 借款人基本信息
    'emp_title',           # 雇主职位
    'emp_length',          # 就业年限
    'home_ownership',      # 住房所有权状态
    'annual_inc',          # 年收入
    'verification_status', # 验证状态
    'issue_d',             # 贷款发放日期（可用，标记贷款开始时间）
    'pymnt_plan',          # 还款计划
    'initial_list_status', # 初始列表状态
    'application_type',    # 申请类型
    'policy_code',         # 政策代码
    
    # 3. 信用历史（申请前的信用状况）
    'dti',                 # 债务收入比
    'delinq_2yrs',         # 过去2年拖欠次数
    'earliest_cr_line',    # 最早信用额度开立日期
    'fico_range_low',      # FICO分数范围低值
    'fico_range_high',     # FICO分数范围高值
    'inq_last_6mths',      # 过去6个月查询次数
    'mths_since_last_delinq',      # 距离上次拖欠的月数
    'mths_since_last_record',      # 距离上次公共记录的月数
    'open_acc',            # 开放信用额度数量
    'pub_rec',             # 公共贬损记录数量
    'revol_bal',           # 循环余额
    'revol_util',          # 循环额度使用率
    'total_acc',           # 信用额度总数
    'collections_12_mths_ex_med',  # 过去12个月催收次数（不含医疗）
    'mths_since_last_major_derog', # 距离上次重大贬损的月数
    'acc_now_delinq',      # 当前拖欠账户数
    'tot_coll_amt',        # 催收总额
    'tot_cur_bal',         # 当前所有账户总余额
    'open_acc_6m',         # 过去6个月开立账户数
    'open_act_il',         # 开放分期付款账户数
    'open_il_12m',         # 过去12个月开立分期付款账户数
    'open_il_24m',         # 过去24个月开立分期付款账户数
    'mths_since_rcnt_il',  # 距离最近分期付款的月数
    'total_bal_il',        # 分期付款总余额
    'il_util',             # 分期付款额度使用率
    'open_rv_12m',         # 过去12个月开立循环账户数
    'open_rv_24m',         # 过去24个月开立循环账户数
    'max_bal_bc',          # 最大银行卡余额
    'all_util',            # 所有账户额度使用率
    'total_rev_hi_lim',    # 循环信贷总限额
    'inq_fi',              # 金融机构查询次数
    'total_cu_tl',         # 信用合作社账户总数
    'inq_last_12m',        # 过去12个月查询次数
    'acc_open_past_24mths',# 过去24个月开立账户数
    'avg_cur_bal',         # 所有账户平均当前余额
    'bc_open_to_buy',      # 银行卡可用额度
    'bc_util',             # 银行卡使用率
    'chargeoff_within_12_mths', # 过去12个月核销数
    'delinq_amnt',         # 拖欠金额
    'mo_sin_old_il_acct',  # 最早分期付款账户月数
    'mo_sin_old_rev_tl_op',# 最早循环账户月数
    'mo_sin_rcnt_rev_tl_op',# 最近循环账户月数
    'mo_sin_rcnt_tl',      # 最近账户月数
    'mort_acc',            # 抵押账户数
    'mths_since_recent_bc',# 距离最近银行卡的月数
    'mths_since_recent_bc_dlq', # 距离最近银行卡拖欠的月数
    'mths_since_recent_inq',    # 距离最近查询的月数
    'mths_since_recent_revol_delinq', # 距离最近循环拖欠的月数
    'num_accts_ever_120_pd',    # 逾期120天以上的账户数
    'num_actv_bc_tl',      # 活跃银行卡账户数
    'num_actv_rev_tl',     # 活跃循环账户数
    'num_bc_sats',         # 满意银行卡账户数
    'num_bc_tl',           # 银行卡账户总数
    'num_il_tl',           # 分期付款账户数
    'num_op_rev_tl',       # 开放循环账户数
    'num_rev_accts',       # 循环账户数
    'num_rev_tl_bal_gt_0', # 余额>0的循环账户数
    'num_sats',            # 满意账户数
    'num_tl_120dpd_2m',    # 过去2个月逾期120天的账户数
    'num_tl_30dpd',        # 逾期30天的账户数
    'num_tl_90g_dpd_24m',  # 过去24个月逾期90天以上的账户数
    'num_tl_op_past_12m',  # 过去12个月开立的账户数
    'pct_tl_nvr_dlq',      # 从未拖欠的账户百分比
    'percent_bc_gt_75',    # 使用率>75%的银行卡百分比
    'pub_rec_bankruptcies',# 公开破产记录数
    'tax_liens',           # 税收留置权数
    'tot_hi_cred_lim',     # 所有账户信贷限额总和
    'total_bal_ex_mort',   # 除抵押外的总余额
    'total_bc_limit',      # 银行卡总限额
    'total_il_high_credit_limit', # 分期付款总信贷限额
    
    # 4. 联合申请人信息（如果是joint application）
    'annual_inc_joint',        # 联合年收入
    'dti_joint',               # 联合债务收入比
    'verification_status_joint', # 联合验证状态
    'sec_app_fico_range_low',  # 第二申请人FICO低值
    'sec_app_fico_range_high', # 第二申请人FICO高值
    'sec_app_earliest_cr_line', # 第二申请人最早信用额度
    'sec_app_inq_last_6mths',   # 第二申请人过去6个月查询
    'sec_app_mort_acc',         # 第二申请人抵押账户
    'sec_app_open_acc',         # 第二申请人开放账户
    'sec_app_revol_util',       # 第二申请人循环使用率
    'sec_app_open_act_il',      # 第二申请人开放分期账户
    'sec_app_num_rev_accts',    # 第二申请人循环账户数
    'sec_app_chargeoff_within_12_mths', # 第二申请人12个月核销
    'sec_app_collections_12_mths_ex_med', # 第二申请人12个月催收
    'sec_app_mths_since_last_major_derog', # 第二申请人距上次贬损月数
]

# ========================================================================
# ❌ POST-LOAN FEATURES (贷款后 - 必须删除，会造成Leakage)
# ========================================================================
post_loan_features = [
    # 1. 还款相关（贷款后才产生）
    'out_prncp',           # 未偿还本金
    'out_prncp_inv',       # 投资者未偿还本金
    'total_pymnt',         # 收到的总还款
    'total_pymnt_inv',     # 投资者收到的总还款
    'total_rec_prncp',     # 收到的本金
    'total_rec_int',       # 收到的利息
    'total_rec_late_fee',  # 收到的滞纳金
    'recoveries',          # 回收金额（违约后）
    'collection_recovery_fee', # 催收费用
    'last_pymnt_d',        # 最后还款日期
    'last_pymnt_amnt',     # 最后还款金额
    'next_pymnt_d',        # 下次还款日期
    'last_credit_pull_d',  # 最后信用查询日期
    'last_fico_range_high',# 最近FICO分数高值
    'last_fico_range_low', # 最近FICO分数低值
    
    # 2. 困难/重组相关（贷款后的事件）
    'hardship_flag',       # 困难标志
    'hardship_type',       # 困难类型
    'hardship_reason',     # 困难原因
    'hardship_status',     # 困难状态
    'deferral_term',       # 延期期限
    'hardship_amount',     # 困难金额
    'hardship_start_date', # 困难开始日期
    'hardship_end_date',   # 困难结束日期
    'payment_plan_start_date', # 还款计划开始日期
    'hardship_length',     # 困难长度
    'hardship_dpd',        # 困难逾期天数
    'hardship_loan_status',# 困难贷款状态
    'orig_projected_additional_accrued_interest', # 预计额外应计利息
    'hardship_payoff_balance_amount', # 困难还清余额
    'hardship_last_payment_amount',   # 困难最后还款额
    
    # 3. 债务清偿相关（违约后的处理）
    'debt_settlement_flag',      # 债务清偿标志
    'debt_settlement_flag_date', # 债务清偿日期
    'settlement_status',         # 清偿状态
    'settlement_date',           # 清偿日期
    'settlement_amount',         # 清偿金额
    'settlement_percentage',     # 清偿百分比
    'settlement_term',           # 清偿期限
]

# ========================================================================
# 🗑️ METADATA/ID FEATURES (元数据 - 需删除，无预测价值)
# ========================================================================
metadata_features = [
    'id',                  # 贷款ID
    'member_id',           # 会员ID
    'url',                 # 贷款URL
    'desc',                # 描述（但保留用于OCEAN提取）
    'zip_code',            # 邮编（太细粒度）
    'addr_state',          # 州（可能有用，但先标记）
]

# ========================================================================
# 🎯 TARGET VARIABLE
# ========================================================================
target_variable = 'loan_status'

# 注意：desc 既是metadata（需删除ID性质），也用于特征工程（OCEAN提取）
# 所以我们会：
# 1. 保留 desc 用于 OCEAN 特征提取
# 2. 提取完 OCEAN 特征后，删除 desc
# 3. metadata_features 中包含 desc，但在特征工程阶段会特殊处理

print(f"✓ PRE-LOAN 特征: {len(pre_loan_features)} 个")
print(f"❌ POST-LOAN 特征: {len(post_loan_features)} 个")
print(f"🗑️ METADATA 特征: {len(metadata_features)} 个")
print(f"🎯 目标变量: 1 个")
print(f"\n总计定义: {len(pre_loan_features) + len(post_loan_features) + len(metadata_features) + 1} 个")

In [ ]:
# Cell 4: 检查分类覆盖率

print("=" * 80)
print("检查特征分类覆盖率")
print("=" * 80)

# 获取数据集中的所有列
all_columns_in_data = set(df.columns)

# 获取我们分类的所有列
all_classified = set(pre_loan_features + post_loan_features + metadata_features + [target_variable])

# 找出未分类的列
unclassified = all_columns_in_data - all_classified

# 找出分类中但数据中不存在的列
not_in_data = all_classified - all_columns_in_data

print(f"\n数据集中的列数: {len(all_columns_in_data)}")
print(f"已分类的列数: {len(all_classified)}")

if unclassified:
    print(f"\n⚠️ 未分类的列 ({len(unclassified)} 个):")
    for col in sorted(unclassified):
        print(f"  - {col}")
else:
    print(f"\n✓ 所有列都已分类")

if not_in_data:
    print(f"\n⚠️ 分类中存在但数据中不存在的列 ({len(not_in_data)} 个):")
    for col in sorted(not_in_data):
        # 检查属于哪个分类
        if col in pre_loan_features:
            category = "PRE-LOAN"
        elif col in post_loan_features:
            category = "POST-LOAN"
        elif col in metadata_features:
            category = "METADATA"
        else:
            category = "TARGET"
        print(f"  - {col} ({category})")
else:
    print(f"\n✓ 所有分类的列都存在于数据中")

In [ ]:
# Cell 5: 创建特征分类 DataFrame

print("=" * 80)
print("创建特征分类报告")
print("=" * 80)

# 创建分类字典
feature_classification = {}

# PRE-LOAN features
for feat in pre_loan_features:
    if feat in df.columns:
        coverage = (df[feat].notna().sum() / len(df)) * 100
        feature_classification[feat] = {
            'category': 'PRE-LOAN',
            'usable': '✅ Yes',
            'leakage_risk': '❌ No',
            'coverage_%': f"{coverage:.2f}%",
            'dtype': str(df[feat].dtype),
            'note': 'Safe to use - known at application time'
        }

# POST-LOAN features
for feat in post_loan_features:
    if feat in df.columns:
        coverage = (df[feat].notna().sum() / len(df)) * 100
        feature_classification[feat] = {
            'category': 'POST-LOAN',
            'usable': '❌ No',
            'leakage_risk': '⚠️ HIGH',
            'coverage_%': f"{coverage:.2f}%",
            'dtype': str(df[feat].dtype),
            'note': 'MUST DELETE - causes data leakage'
        }

# METADATA features
for feat in metadata_features:
    if feat in df.columns:
        coverage = (df[feat].notna().sum() / len(df)) * 100
        note = 'Delete - no predictive value' if feat != 'desc' else 'Keep for OCEAN extraction, then delete'
        feature_classification[feat] = {
            'category': 'METADATA',
            'usable': '❌ No' if feat != 'desc' else '⚠️ Special',
            'leakage_risk': '❌ No',
            'coverage_%': f"{coverage:.2f}%",
            'dtype': str(df[feat].dtype),
            'note': note
        }

# TARGET variable
if target_variable in df.columns:
    coverage = (df[target_variable].notna().sum() / len(df)) * 100
    feature_classification[target_variable] = {
        'category': 'TARGET',
        'usable': '🎯 Target',
        'leakage_risk': 'N/A',
        'coverage_%': f"{coverage:.2f}%",
        'dtype': str(df[target_variable].dtype),
        'note': 'This is what we want to predict'
    }

# 创建 DataFrame
classification_df = pd.DataFrame.from_dict(feature_classification, orient='index')
classification_df.index.name = 'feature_name'
classification_df = classification_df.reset_index()

# 按类别排序
category_order = {'PRE-LOAN': 1, 'POST-LOAN': 2, 'METADATA': 3, 'TARGET': 4}
classification_df['sort_order'] = classification_df['category'].map(category_order)
classification_df = classification_df.sort_values('sort_order').drop('sort_order', axis=1)

print(f"\n特征分类摘要:")
print(classification_df.groupby('category').size())

print(f"\n前20行预览:")
display(classification_df.head(20))

In [ ]:
# Cell 6: 保存特征分类清单

print("=" * 80)
print("保存特征分类清单")
print("=" * 80)

# 1. 保存完整分类
output_file_1 = '../../feature_classification_complete.csv'
classification_df.to_csv(output_file_1, index=False)
print(f"\n✓ 完整分类已保存: {output_file_1}")

# 2. 保存 PRE-LOAN 特征清单（可以使用的）
pre_loan_df = classification_df[classification_df['category'] == 'PRE-LOAN'].copy()
output_file_2 = '../../features_pre_loan.csv'
pre_loan_df.to_csv(output_file_2, index=False)
print(f"✓ PRE-LOAN 特征清单已保存: {output_file_2}")
print(f"  - {len(pre_loan_df)} 个可用特征")

# 3. 保存 POST-LOAN 特征清单（必须删除的）
post_loan_df = classification_df[classification_df['category'] == 'POST-LOAN'].copy()
output_file_3 = '../../features_post_loan.csv'
post_loan_df.to_csv(output_file_3, index=False)
print(f"❌ POST-LOAN 特征清单已保存: {output_file_3}")
print(f"  - {len(post_loan_df)} 个需删除的特征（会造成leakage）")

# 4. 保存 METADATA 特征清单（需删除的）
metadata_df = classification_df[classification_df['category'] == 'METADATA'].copy()
output_file_4 = '../../features_metadata.csv'
metadata_df.to_csv(output_file_4, index=False)
print(f"🗑️ METADATA 特征清单已保存: {output_file_4}")
print(f"  - {len(metadata_df)} 个需删除的特征（无预测价值）")

# 5. 创建简单的特征列表（用于后续代码）
# 只保留特征名
pre_loan_list = pre_loan_df['feature_name'].tolist()
post_loan_list = post_loan_df['feature_name'].tolist()
metadata_list = metadata_df['feature_name'].tolist()

# 保存为Python可读的格式
with open('../../feature_lists.py', 'w') as f:
    f.write("# Feature Lists Generated by 02_feature_selection_and_leakage_check.ipynb\n\n")
    f.write("# PRE-LOAN Features (Safe to use)\n")
    f.write("PRE_LOAN_FEATURES = [\n")
    for feat in pre_loan_list:
        f.write(f"    '{feat}',\n")
    f.write("]\n\n")
    
    f.write("# POST-LOAN Features (MUST DELETE - causes leakage)\n")
    f.write("POST_LOAN_FEATURES = [\n")
    for feat in post_loan_list:
        f.write(f"    '{feat}',\n")
    f.write("]\n\n")
    
    f.write("# METADATA Features (DELETE - no predictive value)\n")
    f.write("METADATA_FEATURES = [\n")
    for feat in metadata_list:
        f.write(f"    '{feat}',\n")
    f.write("]\n\n")
    
    f.write(f"# TARGET Variable\n")
    f.write(f"TARGET_VARIABLE = '{target_variable}'\n")

print(f"\n✓ Python特征列表已保存: feature_lists.py")
print(f"\n所有文件已保存！")

In [ ]:
# Cell 7: 生成最终摘要报告

print("=" * 80)
print("特征分类与 Data Leakage 检查 - 最终报告")
print("=" * 80)

print(f"\n📊 数据集信息:")
print(f"  - 总行数: {len(df):,}")
print(f"  - 总列数: {len(df.columns)}")

print(f"\n📋 特征分类结果:")
print(f"  ✅ PRE-LOAN 特征: {len(pre_loan_df)} 个")
print(f"     - 可以安全使用于模型训练")
print(f"     - 这些信息在贷款申请时就已知")

print(f"\n  ❌ POST-LOAN 特征: {len(post_loan_df)} 个")
print(f"     - 必须删除！会造成 Data Leakage")
print(f"     - 这些信息在贷款发放后才产生")
print(f"     - 使用这些特征会导致模型过拟合")

print(f"\n  🗑️ METADATA 特征: {len(metadata_df)} 个")
print(f"     - 建议删除（除了desc用于OCEAN提取）")
print(f"     - ID字段无预测价值")

print(f"\n  🎯 目标变量: 1 个 ({target_variable})")

print(f"\n⚠️ Data Leakage 风险分析:")
leakage_features = classification_df[classification_df['leakage_risk'] == '⚠️ HIGH']
print(f"  - 发现 {len(leakage_features)} 个高风险特征")
print(f"  - 这些特征在实际应用中无法获得")
print(f"  - 必须在建模前删除")

print(f"\n✅ 建模推荐:")
print(f"  1. 使用 {len(pre_loan_df)} 个 PRE-LOAN 特征")
print(f"  2. 从 desc 字段提取 OCEAN 特征 (5个)")
print(f"  3. 预计最终特征数: {len(pre_loan_df) + 5} 个")
print(f"  4. 目标变量: {target_variable}")

print(f"\n📁 输出文件:")
print(f"  ✓ feature_classification_complete.csv - 完整分类")
print(f"  ✓ features_pre_loan.csv - 可用特征清单")
print(f"  ✓ features_post_loan.csv - 需删除特征清单（Leakage）")
print(f"  ✓ features_metadata.csv - 需删除特征清单（Metadata）")
print(f"  ✓ feature_lists.py - Python特征列表")

print(f"\n🚀 下一步:")
print(f"  1. 运行 03_create_modeling_dataset.ipynb")
print(f"  2. 删除 POST-LOAN 和 METADATA 特征")
print(f"  3. 只保留 PRE-LOAN 特征 + desc + target")
print(f"  4. 创建干净的建模数据集")

print(f"\n" + "=" * 80)
print(f"特征分类完成！")
print("=" * 80)

In [ ]:
# Cell 8: 显示 Leakage 特征的示例数据

print("=" * 80)
print("Data Leakage 示例分析")
print("=" * 80)

print("\n以下是一些典型的 POST-LOAN 特征，它们会造成 data leakage:\n")

# 选择几个典型的leakage特征
leakage_examples = [
    'total_pymnt',
    'total_rec_prncp',
    'total_rec_int',
    'out_prncp',
    'recoveries'
]

# 过滤出存在的特征
leakage_examples = [f for f in leakage_examples if f in df.columns]

if leakage_examples and 'loan_status' in df.columns:
    # 显示这些特征与loan_status的关系
    sample_cols = ['loan_status'] + leakage_examples
    sample_df = df[sample_cols].head(10)
    
    print("\n示例数据（前10行）:")
    display(sample_df)
    
    print("\n⚠️ 为什么这些特征会造成 Leakage?")
    print("\n1. total_pymnt (总还款额):")
    print("   - 只有贷款结束后才知道总共还了多少钱")
    print("   - 如果贷款违约(Charged Off)，total_pymnt会很低")
    print("   - 如果贷款还清(Fully Paid)，total_pymnt会接近贷款金额+利息")
    print("   - 模型会学到：total_pymnt高 ➜ Fully Paid，total_pymnt低 ➜ Charged Off")
    print("   - 但在实际应用时，我们无法在贷款前知道total_pymnt！")
    
    print("\n2. out_prncp (未偿还本金):")
    print("   - 只有在贷款进行中才知道还剩多少本金未还")
    print("   - 如果 out_prncp = 0 ➜ 肯定是 Fully Paid")
    print("   - 如果 out_prncp > 0 ➜ 可能是 Charged Off 或还在还款中")
    
    print("\n3. recoveries (回收金额):")
    print("   - 只有在贷款违约后，通过催收才能回收资金")
    print("   - 如果 recoveries > 0 ➜ 肯定已经违约了")
    print("   - 这是违约后才产生的数据！")
    
    print("\n✅ 正确做法:")
    print("   只使用贷款申请时就已知的信息：")
    print("   - annual_inc (年收入)")
    print("   - dti (债务收入比)")
    print("   - fico_range_low (FICO分数)")
    print("   - emp_length (就业年限)")
    print("   - purpose (贷款目的)")
    print("   - etc.")
else:
    print("\n⚠️ 示例特征在数据中不存在，跳过展示")